# Topic modeling

In [47]:
import sys
sys.path.append('..')
from ballitoreproject import *
from ballitoreproject.topicmodels import *


In [3]:
# get all data (metadata plus letters)
df = get_data()
df

,sender,recipient,date,location from,location to,notes,box,datetime,year,decade,is_journal,txt,supplemental_ids,num_letters,num_words
id,,,,,,,,,,,,,,,
mss4-b1-f7-001-176,William Penn,Unknown 1.1,2/10/1690,London,,copy; p. 2,1,1690-02-10,1690,1690,False,,mss4-b1-f7-001-177,1,0
mss4-b1-f3-005,Abraham Shackleton I,Richard Shackleton,1744,,Dublin,extract,1,1744-07-13,1744,1740,False,"P.7\t“Keep thy mind as quiet as possible, so t...",,1,37
mss4-b1-f7-001-3,Sophia Hume,Anne Hyam,6/1/1748,Philadelphia,,; p. 6,1,1748-06-01,1748,1740,False,,mss4-b1-f7-001-30,1,0
mss4-b1-f3-005-006,Abraham Shackleton I,Richard Shackleton,1748,,Dublin,,1,1748-07-13,1748,1740,False,"“We are very desirous of seeing thee at home, ...",,1,173
mss4-b1-f7-001-15,Sophia Hume,J. P.,5/19/1750,London,,,1,1750-05-19,1750,1750,False,,,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
consensus_text_90345943,,,,,,,14,NaT,0,0,True,"was no company here to day, and\neven Charlie ...",,1,74
consensus_text_90345944,,,,,,,14,NaT,0,0,True,"dinner, Papa, Ned, and the\nrest came out to t...",,1,69
consensus_text_90345945,,,,,,,14,NaT,0,0,True,[deletion]16[/deletion] Papa did not go to tow...,,1,71


In [4]:
df.sender.value_counts().head(25)

sender
                               2206
Richard Shackleton              349
Sarah Leadbeater Barrington     200
William Rayner                   99
Mary Shackleton Leadbeater       74
Margaret Shackleton Grubb        67
Juliet Smith                     51
Selina Barrington                46
Samuel Haughton                  44
Lydia J. Leadbeater Fisher       44
Edward Barrington                42
Thomas Fisher                    42
Benjamin Haughton                42
Sarah Shackleton                 34
Sarah Tuke Grubb                 34
S. Hancock                       32
William Leadbeater               31
Simeon Wilkinson                 27
James J. Fisher                  27
Frances Smith                    25
Elizabeth Leadbeater             23
M. J. Barrington                 23
Elizabeth Dale                   22
Emily Malone                     22
Elizabeth Clibborn               21
Name: count, dtype: int64

In [5]:
# decide on a sample!
df_smpl = df.query('txt!=""')  # only texts (for topic modeling)
df_smpl = df_smpl.query('sender!=""')  # only rows for which we have sender
# df_smpl = df_smpl.query('sender!="" & recipient!=""')  # only rows for which we have sender and recipient
# df_smpl = df_smpl.query('is_journal==True')  # just the journals
# df_smpl = df_smpl.query('sender=="Mary Shackleton Leadbeater"')  # just MSL
# df_smpl = df_smpl.query('sender=="Mary Shackleton Leadbeater" or sender=="Sarah Shackleton"')  # just MSL or SS
df_smpl

,sender,recipient,date,location from,location to,notes,box,datetime,year,decade,is_journal,txt,supplemental_ids,num_letters,num_words
id,,,,,,,,,,,,,,,
mss4-b1-f3-005,Abraham Shackleton I,Richard Shackleton,1744,,Dublin,extract,1,1744-07-13,1744,1740,False,"P.7\t“Keep thy mind as quiet as possible, so t...",,1,37
mss4-b1-f3-005-006,Abraham Shackleton I,Richard Shackleton,1748,,Dublin,,1,1748-07-13,1748,1740,False,"“We are very desirous of seeing thee at home, ...",,1,173
mss4-b1-f3-007,Richard Shackleton,Abraham Shackleton I,11/1/1755,Dublin,,,1,1755-11-01,1755,1750,False,P. 25\t\t\t\t\t\t“Dublin 1st of Eleventhmonth ...,,1,73
mss4-b1-f3-008,Abraham Shackleton I,Richard Shackleton,6/2/1760,London,,,1,1760-06-02,1760,1760,False,"P. 53\t\t\t\t\t\t“London, 2nd of Sixthmonth 17...",,1,43
mss4-b1-f3-008-010,Abraham Shackleton I,Richard Shackleton,9/18/1762,Cooladine,,,1,1762-09-18,1762,1760,False,To the memorable visit paid to the meetings fo...,,1,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mss4-b12-f63-001,Cousin,GL,,,,"essay on ""The Beauties of Nature""",12,NaT,0,0,False,The Beauties of Nature\nWhat is more beautiful...,,1,164
mss4-b12-f63-002,WL?,,,,,,12,NaT,0,0,False,On seeing your Bror yesterday I presume\nyou a...,,1,120
mss4-b12-f63-003,WL?,,,,,includes poem draft in ML's handwriting,12,NaT,0,0,False,Thursday - I received your Enclosure\nand than...,,1,90


In [6]:
from pprint import pprint
pprint(dict(df_smpl.iloc[0]))

{'box': 1,
 'date': 1744,
 'datetime': Timestamp('1744-07-13 00:00:00'),
 'decade': 1740,
 'is_journal': False,
 'location from': '',
 'location to': 'Dublin',
 'notes': 'extract',
 'num_letters': 1,
 'num_words': 37,
 'recipient': 'Richard Shackleton',
 'sender': 'Abraham Shackleton I',
 'supplemental_ids': '',
 'txt': 'P.7\t“Keep thy mind as quiet as possible, so that thou mayst have '
        'benefit of this little tour. I desire thou mayst have thy eye to Him '
        'that hath preserved, in some degree, him that writeth, and is his '
        'chiefest joy. Next to that, my joy is that mine may walk acceptably '
        'before Him. Durable riches, I know, and honours, are in his '
        'right-hand, which He dispenses impartially, in his own fit time.”',
 'year': 1744}


In [7]:
tokenize(df_smpl.iloc[0].txt)

['keep',
 'thy',
 'mind',
 'quiet',
 'possible',
 'thou',
 'mayst',
 'benefit',
 'little',
 'tour',
 'desire',
 'thou',
 'mayst',
 'thy',
 'eye',
 'hath',
 'preserved',
 'degree',
 'writeth',
 'chiefest',
 'joy',
 'next',
 'joy',
 'mine',
 'may',
 'walk',
 'acceptably',
 'durable',
 'riches',
 'know',
 'honours',
 'right',
 'hand',
 'dispenses',
 'impartially',
 'fit',
 'time']

In [8]:
stopwords = get_stopwords()
stopwords|={'us'}
# stopwords

In [9]:
# start a topic model
mdl = tp.LDAModel(k=50)

# loop over dataframe
for id,row in df_smpl.iterrows():
    # get tokens (string of letter to )
    tokens = tokenize(row.txt, stopwords=stopwords)
    # add document
    mdl.add_doc(tokens)

# train model
mdl.train(100, show_progress=True)

# show summary of results
mdl.summary(topic_word_top_n=25)

Iteration: 100%|██████████| 100/100 [00:01<00:00, 93.37it/s, LLPW: -9.016163]


<Basic Info>
| LDAModel (current version: 0.12.7)
| 1931 docs, 453113 words
| Total Vocabs: 28715, Used Vocabs: 28715
| Entropy of words: 7.97182
| Entropy of term-weighted words: 7.97182
| Removed Vocabs: <NA>
|
<Training Info>
| Iterations: 100, Burn-in steps: 0
| Optimization Interval: 10
| Log-likelihood per word: -8.99390
|
<Initial Parameters>
| tw: TermWeight.ONE
| min_cf: 0 (minimum collection frequency of words)
| min_df: 0 (minimum document frequency of words)
| rm_top: 0 (the number of top words to be removed)
| k: 50 (the number of topics between 1 ~ 32767)
| alpha: [0.1] (hyperparameter of Dirichlet distribution for document-topic, given as a single `float` in case of symmetric prior and as a list with length `k` of `float` in case of asymmetric prior.)
| eta: 0.01 (hyperparameter of Dirichlet distribution for topic-word)
| seed: 2143498676 (random seed)
| trained in version 0.12.7
|
<Parameters>
| alpha (Dirichlet prior on the per-document topic distributions)
|  [0.06382

In [10]:
topic_term_dists = pd.DataFrame([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
topic_term_dists.columns = list(mdl.used_vocabs)
topic_term_dists

,thy,dear,thee,thou,may,much,would,well,love,time,...,obligatns,rems,anyone's,chickens,ballylea,obscuring,granny,wel,snooks,boundaries
0,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,...,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06,3.869744e-06
1,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06,1.047493e-03,1.361427e-03,1.046446e-06,1.046446e-06,1.109338e-02,...,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06,1.046446e-06
2,9.417836e-07,2.740685e-02,9.417836e-07,9.417836e-07,4.294628e-02,8.382816e-03,9.417836e-07,2.449579e-03,1.384516e-02,9.417836e-07,...,9.417836e-07,9.417836e-07,9.417836e-07,9.417836e-07,9.417836e-07,9.417836e-07,9.417836e-07,9.417836e-07,9.417836e-07,9.417836e-07
3,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,1.903227e-02,3.069227e-06,3.069227e-06,6.755367e-03,...,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06,3.069227e-06
4,1.508717e-06,1.508717e-06,1.508717e-06,1.508717e-06,2.868070e-03,9.959039e-03,1.508717e-06,1.508717e-06,1.508717e-06,1.508717e-06,...,1.508717e-06,1.523804e-04,1.508717e-06,1.508717e-06,1.508717e-06,1.508717e-06,1.508717e-06,1.508717e-06,1.508717e-06,1.508717e-06
5,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,2.148465e-02,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,...,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06,3.025581e-06
6,7.165813e-07,7.165813e-07,7.165813e-07,7.165813e-07,1.590882e-02,1.354410e-02,7.165813e-07,1.906178e-02,7.165813e-07,7.165813e-07,...,7.165813e-07,7.165813e-07,7.165813e-07,7.165813e-07,7.165813e-07,7.165813e-07,7.165813e-07,7.165813e-07,7.165813e-07,7.165813e-07
7,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,1.051768e-02,2.564662e-06,1.385174e-02,...,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06,2.564662e-06
8,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,...,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06,3.568688e-06
9,1.109613e-06,1.109613e-06,1.109613e-06,1.221684e-03,1.109613e-06,1.109613e-06,1.109613e-06,1.054243e-02,2.109374e-03,1.109613e-06,...,1.109613e-06,1.109613e-06,1.109613e-06,1.109613e-06,1.109613e-06,1.109613e-06,1.109613e-06,1.109613e-06,1.109613e-06,1.109613e-06


In [11]:
top_topics_for_abolition_keyword = topic_term_dists.sort_values('abolition',ascending=False).head(5)
top_topics_for_abolition_keyword

,thy,dear,thee,thou,may,much,would,well,love,time,...,obligatns,rems,anyone's,chickens,ballylea,obscuring,granny,wel,snooks,boundaries
19,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,...,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
38,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,...,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007,0.000007
0,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004
29,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004
8,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,...,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004,0.000004


In [12]:
top_abolition_topic_ids = top_topics_for_abolition_keyword.index

top_topics_for_abolition_keyword.T.sort_values(top_abolition_topic_ids[0],ascending=False).head(100).index

Index(['read', 'great', 'received', 'books', 'yet', 'book', 'pleasure', 'many',
       'met', 'excellent', 'please', 'interesting', 'man', 'acquainted',
       'found', 'society', 'effect', 'letter', 'history', 'hancock', 'lately',
       'different', 'forward', 'high', 'visit', 'character', 'names',
       'country', 'success', 'copies', 'education', 'knowledge', 'introduced',
       'learn', 'straw', 'paper', 'manners', 'established', 'among', 'person',
       'persons', 'extracts', 'regret', 'receive', 'especially', 'learned',
       'attached', 'schools', 'system', 'gentleman', 'literary', 'subject',
       'rejoice', 'advantage', 'instruction', 'characters', 'therefore',
       'form', 'old', 'printing', 'short', 'promises', 'proper', 'trade',
       'prospect', 'pen', 'relative', 'effects', 'return', 'latin', 'possess',
       'merit', 'requested', 'street', 'tour', 'certain', 'scotch', 'article',
       'subscribers', 'review', 'principles', 'produce', 'necessity',
       'corre

In [13]:
top_topics_for_abolition_keyword.T.sort_values(top_abolition_topic_ids[1],ascending=False).head(100).index

Index(['fitzsimons', 'que', 'tore', 'harmony', 'majesty', 'memorial', 'ditto',
       'assignmt', 'dans', 'quakers', 'grand', 'une', 'con', 'est', 'grows',
       'rocks', 'des', 'loyal', 'geo', 'les', 'notre', 'pas', 'laughed',
       'count', 'help', 'highlands', 'farewell', 'mon', 'diogenes',
       'principles', 'king', 'philosophers', 'gives', 'mea', 'chere', 'body',
       'edict', 'broder', 'vergennes', 'wave', 'marie', 'mois', 'qu'il',
       'address'd', 'acts', 'non', 'springmt', 'billet', 'coat', 'judgmt',
       'soil', 'monarch', 'françois', 'harm', 'vous', 'pollux', 'jour', 'nous',
       'avec', 'agst', 'ingenious', 'langue', 'sera', 'tous', 'judt', 'usury',
       'berur', 'knavery', 'marvellously', 'porte', 'hats', 'carving',
       'consequent', 'comme', 'gems', 'roman', 'proposals', 'motives', 'aussi',
       'duchess', 'agent', 'talents', 'sentence', 'street', 'awl',
       'subscriber', 'wld', 'honestly', 'greene', 'souhaite', 'stray',
       'wither', 'beneficence

In [14]:
doc_topic_scores_df=pd.DataFrame([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_scores_df.index = df_smpl.index
doc_topic_scores_df


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
id,,,,,,,,,,,,,,,,,,,,,
mss4-b1-f3-005,0.001295,0.003605,0.006860,0.001887,0.004319,0.001245,0.027529,0.001413,0.021544,0.004493,...,0.009548,0.024864,0.107496,0.188715,0.003348,0.051858,0.010602,0.006153,0.011600,0.052905
mss4-b1-f3-005-006,0.000344,0.027944,0.018016,0.000502,0.028134,0.000331,0.007323,0.005773,0.000334,0.001195,...,0.013334,0.055187,0.077168,0.109567,0.000891,0.100146,0.013614,0.017828,0.003086,0.100425
mss4-b1-f3-007,0.000748,0.013808,0.179837,0.001091,0.002496,0.000720,0.015909,0.000817,0.000725,0.002597,...,0.028968,0.002644,0.062123,0.062160,0.001935,0.018244,0.017852,0.003556,0.006704,0.112648
mss4-b1-f3-008,0.001154,0.003214,0.024203,0.019769,0.003850,0.001110,0.096889,0.001260,0.001119,0.004006,...,0.008512,0.004079,0.150091,0.005455,0.002985,0.010057,0.009452,0.005485,0.010341,0.065251
mss4-b1-f3-008-010,0.000332,0.000924,0.048563,0.000484,0.006307,0.000319,0.017458,0.000362,0.000322,0.032355,...,0.002448,0.001173,0.095161,0.048373,0.000858,0.002892,0.007918,0.022379,0.013374,0.075967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mss4-b12-f63-001,0.000362,0.040715,0.007591,0.000528,0.001207,0.000348,0.081440,0.000395,0.085438,0.001256,...,0.002670,0.012624,0.126488,0.001711,0.000936,0.003154,0.002964,0.001720,0.156401,0.026137
mss4-b12-f63-002,0.068515,0.001343,0.002556,0.038499,0.062083,0.000464,0.055612,0.015645,0.000468,0.016793,...,0.109387,0.001705,0.002256,0.002280,0.001247,0.004203,0.117338,0.055207,0.049677,0.049949
mss4-b12-f63-003,0.010400,0.001737,0.003306,0.000909,0.002081,0.020153,0.003489,0.000681,0.000605,0.031494,...,0.102363,0.002205,0.002917,0.042053,0.011390,0.015212,0.014885,0.051846,0.103352,0.005941


In [15]:
top_abolition_topic_documents = doc_topic_scores_df[top_abolition_topic_ids[0]].sort_values(ascending=False).head(25).index
top_abolition_topic_documents

Index(['mss4-b7-f8-052', 'mss4-b7-f8-052', 'mss4-b11-f42-001',
       'mss4-b5-f50-011', 'mss4-b6-f3-023', 'mss4-b5-f50-010',
       'mss4-b10-f16-004', 'mss4-b10-f28-001', 'mss4-b5-f30-001',
       'mss4-b6-f3-027', 'mss4-b6-f4-007', 'mss4-b6-f31-001', 'mss4-b9-f7-022',
       'mss4-b10-f16-002', 'mss4-b11-f42-003', 'mss4-b9-f7-022',
       'mss4-b6-f3-012', 'mss4-b6-f3-001', 'mss4-b5-f50-020',
       'mss4-b7-f20-017', 'mss4-b7-f8-068', 'mss4-b5-f50-008',
       'mss4-b7-f8-055', 'mss4-b6-f3-020', 'mss4-b5-f50-002'],
      dtype='object', name='id')

In [16]:
df_smpl.loc[top_abolition_topic_documents[0]]

,sender,recipient,date,location from,location to,notes,box,datetime,year,decade,is_journal,txt,supplemental_ids,num_letters,num_words
id,,,,,,,,,,,,,,,
mss4-b7-f8-052,William Rayner,Mary Shackleton Leadbeater,1/19/1802,Waterford,Ballitore,,7,1802-01-19,1802,1800,False,Waterford Jany 19th 1802\nMy dear Cousin\nthe ...,,1,334
mss4-b7-f8-052,William Rayner,Mary Shackleton Leadbeater,5/26/1893,Waterford,Ballitore,,7,1893-05-26,1893,1890,False,Waterford Jany 19th 1802\nMy dear Cousin\nthe ...,,1,334


In [17]:
print(df_smpl.loc[top_abolition_topic_documents[4]].txt)

Lisburn Aug. 16th. 1812
My dear Friend
Thy last letter has lain in my
writing desk six weeks unanswered. Shall
I at once confess that my reason for not writing [insertion]sooner[/insertion]
was because thou mentioned in thy letter that
thou intended putting my letters with those
of thy other correspondents into the form of a
book. I indeed felt vexed that my nonsensical
scrawls should be seen by any one except
thy self. When I sit down to write I do not
take time to consider about the style &c, I
merely think that my friend will be gratified
by hearing from me, & that she will
kindly overlook all defects in my letter. Indeed
it is not often that I can get time to write
a letter without many interruptions, unless
I write when all the family are in bed. Now
do, my dear Friend, let me entreat thee not
to shew my letters, throw them into the fire,
or into a lumber drawer, but do not, ah do not
let them be shewn with the letters of thy other
correspondents. I am conscious that I cannot
writ

In [18]:
from IPython.display import HTML

topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists,
    doc_topic_dists,
    doc_lengths,
    vocab,
    term_frequency,
    start_index=0,
    sort_topics=False
)
pyLDAvis.save_html(prepared_data, 'lda.html')
HTML(filename='lda.html')

In [19]:
all_data=df.join(doc_topic_scores_df)
all_data

,sender,recipient,date,location from,location to,notes,box,datetime,year,decade,...,40,41,42,43,44,45,46,47,48,49
id,,,,,,,,,,,,,,,,,,,,,
mss4-b1-f7-001-176,William Penn,Unknown 1.1,2/10/1690,London,,copy; p. 2,1,1690-02-10,1690,1690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mss4-b1-f3-005,Abraham Shackleton I,Richard Shackleton,1744,,Dublin,extract,1,1744-07-13,1744,1740,...,0.009548,0.024864,0.107496,0.188715,0.003348,0.051858,0.010602,0.006153,0.011600,0.052905
mss4-b1-f7-001-3,Sophia Hume,Anne Hyam,6/1/1748,Philadelphia,,; p. 6,1,1748-06-01,1748,1740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mss4-b1-f3-005-006,Abraham Shackleton I,Richard Shackleton,1748,,Dublin,,1,1748-07-13,1748,1740,...,0.013334,0.055187,0.077168,0.109567,0.000891,0.100146,0.013614,0.017828,0.003086,0.100425
mss4-b1-f7-001-15,Sophia Hume,J. P.,5/19/1750,London,,,1,1750-05-19,1750,1750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
consensus_text_90345943,,,,,,,14,NaT,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consensus_text_90345944,,,,,,,14,NaT,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consensus_text_90345945,,,,,,,14,NaT,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
pd.options.display.max_colwidth=None
df_topics_by_sender = df[['sender']].join(doc_topic_scores_df).groupby('sender').mean(numeric_only=True)
df_topics_by_sender = df_topics_by_sender.T.fillna(0)
df_topics_by_sender = df_topics_by_sender[[c for c in df.sender.value_counts().index if c]]
df_topics_by_sender

sender,Richard Shackleton,Sarah Leadbeater Barrington,William Rayner,Mary Shackleton Leadbeater,Margaret Shackleton Grubb,Juliet Smith,Selina Barrington,Samuel Haughton,Lydia J. Leadbeater Fisher,Edward Barrington,...,Anonymous,Susanna Wright,Job Scott,Anne Tuke,Jane Thomas,James Pike,Helen Thomas,A. Wilson,Thomas Wray,S. T.
0,0.001311,0.001621,0.010844,0.001929,0.000280,0.001676,0.000607,0.009460,0.002091,0.005998,...,0.000522,0.000540,0.0,0.002339,0.000462,0.149376,0.000558,0.000482,0.024429,0.000993
1,0.007299,0.015204,0.007438,0.010597,0.011740,0.008851,0.046751,0.009197,0.002367,0.015578,...,0.001453,0.001502,0.0,0.006512,0.008516,0.003758,0.001555,0.001343,0.000857,0.002764
2,0.034401,0.013440,0.012854,0.013068,0.055013,0.011894,0.005936,0.017426,0.005543,0.010082,...,0.002765,0.070490,0.0,0.012391,0.038601,0.007150,0.020458,0.032793,0.016104,0.005260
3,0.004460,0.004732,0.006165,0.006749,0.001880,0.004729,0.004183,0.012803,0.000447,0.009289,...,0.049825,0.000786,0.0,0.003409,0.000673,0.001967,0.000814,0.000703,0.005273,0.001447
4,0.010779,0.012730,0.023237,0.008632,0.009949,0.044297,0.003810,0.017393,0.001024,0.008370,...,0.001741,0.001799,0.0,0.007800,0.008770,0.046794,0.019362,0.001609,0.044444,0.003311
5,0.001290,0.001867,0.002971,0.006268,0.001738,0.007098,0.000588,0.008656,0.000295,0.001861,...,0.000502,0.000519,0.0,0.002250,0.000444,0.001298,0.000537,0.023142,0.000296,0.109838
6,0.053807,0.012881,0.029564,0.026504,0.047127,0.009095,0.010197,0.025698,0.005905,0.012678,...,0.002918,0.003017,0.0,0.013078,0.017043,0.007547,0.003123,0.048053,0.001722,0.021106
7,0.000806,0.003909,0.001968,0.001200,0.000593,0.003396,0.001150,0.012401,0.001227,0.004316,...,0.000570,0.000589,0.0,0.002552,0.000504,0.001473,0.000609,0.023204,0.000336,0.001083
8,0.004549,0.001207,0.011630,0.002114,0.001387,0.007036,0.000509,0.006100,0.000298,0.002516,...,0.000506,0.000523,0.0,0.038912,0.000447,0.001308,0.000541,0.000468,0.005123,0.000962
9,0.015180,0.004387,0.012705,0.007847,0.025792,0.006887,0.003465,0.022972,0.001957,0.010222,...,0.001811,0.027234,0.0,0.008116,0.016064,0.068123,0.045686,0.009233,0.001068,0.112328


In [44]:
topic_names = {i: f'T{i}: ' + ' '.join(x for x,y in mdl.get_topic_words(i, 10)) for i in range(mdl.k)}
topic_names

{0: 'T0: miss esqr thank harper beg told assure sincere good regards',
 1: 'T1: morning went get night home got oclock saw see could',
 2: 'T2: may father dear great good desire children often times hope',
 3: 'T3: would nearly add variety except know boys generally gave kind',
 4: 'T4: old good ever still never health kind truly must death',
 5: 'T5: magazine belfast lisburn much next irish letter admire indeed think',
 6: 'T6: good believe meeting well think may place much perhaps thine',
 7: 'T7: frds wife frd place bro time shall last self left',
 8: 'T8: eyre capt shall men regt ship war tom board property',
 9: 'T9: wife also near friends several good acceptable kind letter satisfaction',
 10: 'T10: carriage fine horses tell many large think going whole side',
 11: 'T11: thy family children cousin son husband daughter account may present',
 12: 'T12: love may glad thy last shall thee health relations many',
 13: 'T13: friend last sent never opinion tell cannot miss poem thank',
 

In [45]:
df_cmp = pd.DataFrame()
df_cmp['mary'] = df_topics_by_sender['Mary Shackleton Leadbeater'].sort_values(ascending=False)
df_cmp['sarah'] = df_topics_by_sender['Sarah Leadbeater Barrington'].sort_values(ascending=False)
df_cmp['mary - sarah'] = df_cmp['sarah'] - df_cmp['mary']
df_cmp['topic'] = [' '.join(x for x,y in mdl.get_topic_words(i, 25)) for i in df_cmp.index]
df_cmp.sort_values('mary - sarah')

,mary,sarah,mary - sarah,topic
26,0.054938,0.005758,-0.049181,alas thought quite indeed went hear agreeable dear days thou said upon fire good bed many away remains next saw shall attention nurse way scene
22,0.037131,0.003991,-0.033140,upon little sea heart tho' hand side fruit tears near eye making neither met fair beautiful fell bathing loss present scene take land find found
46,0.054375,0.022534,-0.031841,dear sister mother love letter poor kind believe brother affectionate hope know aunt health obliged family great much wish received hear favour heart loss last
29,0.021043,0.000582,-0.020461,come old last made grief making away joy cheer went fall black philip gentlemen side friends story remember gives dreadful says comes loud tears purple
40,0.029057,0.014104,-0.014953,friend friends feel dear present may much many must subject sincere state world rather happy particular occasion every feelings however account particularly wish late others
18,0.018550,0.003754,-0.014795,much though even one reading read written least pleasure without warm others public common till surely another ideas resign taste easily delighted eyes tongue complaint
6,0.026504,0.012881,-0.013623,good believe meeting well think may place much perhaps thine suppose thought seems rather seemed pretty agreeable come returned something morning better seem man sometimes
41,0.018954,0.008499,-0.010455,cousin thy thee dear sister love father hear well brother wife see letter mother one son glad friends kind pretty know daughter since health give
49,0.026105,0.016962,-0.009143,little mind time yet many may one often indeed without another best life way times part well great cause made felt hand others affection place
19,0.016906,0.007943,-0.008963,read great received books yet book pleasure many met excellent please interesting man acquainted found society effect letter history hancock lately forward high different visit


In [46]:
from ballitoreproject.networks import get_correspondence_network, plot_network
G=get_correspondence_network()
for node in list(G.nodes()):
    if node in df_topics_by_sender.columns:
        top_topics = df_topics_by_sender[node].sort_values(ascending=False).index[:1]
        for top_topic in top_topics:
            G.add_edge(node, topic_names[top_topic], relationship='wrote_about')
plot_network(G, edge_color='relationship')

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'content': …